In [15]:
#change ovito export xyz file to RBS simulation input data file
#ovito export xyz file format is :
#25200000
#Lattice="0.0 0.0 5824.86343385 0.0 211.4 0.0 215.0 0.0 0.0" Origin="-86.0 -90.6 -2900.7819900573" Properties=pos:R:3:species:S:1:id:I:1
#-79.5354212843 -89.9262116686 -2899.4576111988 Ga 1
#-80.8037735621 -88.2192198017 -2890.5819782791 Ga 2
#-85.4477714422 -86.5041908662 -2899.4191513497 Ga 3
#RBS data file example:
#25200000
#Lattice="215.00000 0.00000 0.00000 0.00000 214.4 0.00000 0.00000 0.00000 5824.86343385"
#Ga -79.5354212843 -89.9262116686 -2899.4576111988  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1


In [8]:
import pandas as pd
import numpy as np
import os
import re

In [31]:

# Replace 'your_file.txt' with the actual path to your file
file_path = '/home/heruhe/Desktop/Ga2o3/RBS/gamma110/gamma110.in'
file_path='/home/heruhe/Desktop/Ga2o3/combine_cells/cascade_npt/1000PKA'
file_path='/home/heruhe/Desktop/Ga2o3/RBS/Base/600PKAgamma-Ga2O3'
try:
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i < 3:
                print("Line {}: {}".format(i + 1, line.strip())) # strip() removes leading/trailing whitespace
                
            else:
                break
except FileNotFoundError:
    print("File not found:", file_path)
except Exception as e:
    print("An error occurred:", str(e))

Line 1: 207360
Line 2: Lattice="142.4941763958 0.0 0.0 0.0 123.3927236563 0.0 0.0 0.0 130.9280109931" Properties=pos:R:3:species:S:1
Line 3: 0.8397134867 1.9968356009 3.4645878827 2


In [32]:
df=pd.read_csv(file_path,skiprows=2,header=None,sep='\s+')
df.columns=['x','y','z','type']
df

,x,y,z,type
0,0.839713,1.996836,3.464588,2
1,2.389882,2.923051,0.735888,2
2,0.793794,0.648964,1.167261,2
3,0.660871,3.775283,3.961208,1
4,2.283852,0.944595,2.552423,1
...,...,...,...,...
207355,137.319283,120.213336,127.170545,2
207356,137.632434,121.973230,129.741192,2
207357,135.965937,123.076340,127.322592,2
207358,140.334559,121.725358,129.683971,2


In [39]:

# Function to convert OVITO XYZ format to RBS format for a single line
#Move the cell origin to zero
def convert_xyz_to_rbs(line,bx,by,bz):
    parts = line.split()
    if len(parts) == 4:
        x, y, z,id_val = parts
        x=float(x)-bx
        y=float(y)-by
        z=float(z)-bz
        s=''
        id_val=int(id_val)
        if id_val==1:
            s='GaI'
        if id_val==2:
            s='OI'
        if id_val==3:
            s='OI'
        if id_val==4:
            s='OII'
        if id_val==5:
            s='OIII'
        
        return f"{s} {x} {y} {z} {id_val}\n"
        
    else:
        return None
    
    



# Replace 'input.xyz' with your input XYZ file and 'output.rbs' with your desired output RBS file
input_file = file_path
output_file = '/home/heruhe/Desktop/Ga2o3/RBS/Base/600PKAgamma.in'

try:
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Copy the first two lines as is (header lines)
        for i, line in enumerate(infile):
            # Copy the first two lines as is (header lines)
            if i ==0:
                #print(line)
                outfile.write(line)
            if i==1:
                # Define a regular expression pattern to match 'Lattice="..."' and capture the content inside the quotes
                pattern = r'Lattice="([^"]*)"'
                # Define a regular expression pattern to match 'Origin="..."' and capture the content inside the quotes
                pattern2 = r'Origin="([^"]*)"'

                # Use re.search() to find the pattern in the input string
                match = re.search(pattern, line)
                match2 = re.search(pattern2, line)
                print(match2)
                # Check if a match was found
                if match or match2:
                    lattice_value =match.group(1) 
                    numbers = [float(x) for x in lattice_value.split()]
                    if match2!=None:
                        origin_value =match2.group(1) 
                        numbers2 = [float(x) for x in origin_value.split()]
                        print(numbers2)
                    else:
                        numbers2=[0,0,0]
                    # Create a 3x3 matrix from the list of numbers
                    matrix = np.array(numbers).reshape(3, 3)
                    
                    # Print the resulting matrix
                    print(matrix)
                    y=matrix[1][1]
                    z=matrix[2][2]
                    x=matrix[0][0]
                    move_z=numbers2[2]
                    move_y=numbers2[1]
                    move_x=numbers2[0]
                    
                    Lattice='Lattice="'+str(x)+' 0.00000'+' 0.00000'+' 0.00000 '+str(y)+' 0.00000'+' 0.00000'+' 0.00000 '+str(z)+'"\n'
                    outfile.write(Lattice)
                else:
                    print("Lattice not found in the input string.")
            if i>1:
                convert_line=convert_xyz_to_rbs(line,move_x,move_y,move_z)
                outfile.write(convert_line)
                
except FileNotFoundError:
    print("File not found:", input_file)
except Exception as e:
    print("An error occurred:", str(e))

print("Conversion complete.")

None
[[142.4941764    0.           0.        ]
 [  0.         123.39272366   0.        ]
 [  0.           0.         130.92801099]]
Conversion complete.
